In [41]:
import os
import datetime
from datetime import timedelta
import csv
import time
import re
import string
import requests
import numpy as np
from urllib.parse import urlencode
import selenium
from selenium                         import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import   By 
from selenium.webdriver.firefox.options  import   Options 
from selenium.webdriver.support import   expected_conditions  as   EC 
from selenium.webdriver.support.wait  import   WebDriverWait
from selenium.common.exceptions       import NoSuchElementException as NSE #gestione delle eccezioni
from selenium.common.exceptions       import TimeoutException as TOE
from selenium.common.exceptions       import StaleElementReferenceException as SERE
from selenium.common.exceptions       import ElementClickInterceptedException as ECIE
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [42]:

cities=['L\'Aquila','Teramo','Pescara','Chieti',
        'Potenza','Matera',
        'Cosenza','Catanzaro, Calabria, Italia','Crotone','Vibo Valentia','Reggio Calabria',
        'Caserta','Benevento','Avellino','Salerno','Napoli',
        'Piacenza','Parma','Reggio Emilia','Modena','Ferrara','Ravenna','Bologna','Rimini','Forlì',
        'Udine','Gorizia','Trieste','Pordenone',
        'Viterbo','Rieti','Latina','Frosinone','Roma',
        'Imperia','Savona','La Spezia','Genova',
        'Bergamo','Brescia','Como','Cremona','Lecco','Lodi','Mantova','Milano','Varese','Monza','Sondrio','Pavia',
        'Ancona','Macerata','Ascoli Piceno','Fermo','Pesaro',
        'Campobasso','Isernia',
        'Asti','Alessandria, Piemonte, Italia','Biella','Vercelli','Novara','Cuneo','Torino','Verbania',
        'Foggia','Taranto','Brindisi','Lecce','Bari','Andria',
        'Sassari','Nuoro','Oristano','Cagliari','Carbonia',
        'Trapani','Agrigento','Caltanissetta','Enna','Ragusa','Siracusa','Palermo','Messina','Catania',
        'Lucca','Pistoia','Livorno','Pisa','Arezzo','Siena','Grosseto','Prato, Toscana, Italia','Firenze','Massa, Toscana, Italia',
        'Bolzano','Trento',
        'Perugia','Terni',
        'Aosta',
        'Verona','Vicenza','Belluno','Treviso','Padova','Rovigo','Venezia'
        ]

#cities=['Catanzaro, Calabria, Italia','Crotone']
attesaCorta=3 #variabile attesa esplicita caricamento pagina
#ignoredExceptions=(NSE, SERE, TOE, ECIE)

In [43]:
def preparaDriver():
    options = Options()
    options.headless = False  #modalita senza interfaccia grafica con false
    options.add_argument('--lang=it') #imposto la lingua del browser in italiano
    options.set_preference("permissions.default.image", 2) #disablita lo scaricamento delle immagini
    driver = Firefox(options=options) #invoco il webdriver di firefox geckodriver in path
    return driver


def scrapeNumeroStruttureDisponibili(driver, dataOdierna, numstruttureTrovate):  #visita le singole pagine degli hotel 
    hotel=dict()
    hotel['citta']=stringaCitta  #nome della città
    hotel['data odierna']=dataOdierna
    hotel['disponibilita odierna']=numstruttureTrovate
    print(stringaCitta)
    print(numstruttureTrovate)
    return hotel

if __name__ == '__main__':
#def main():
    try:
        alloggiCitta=list()
        date = datetime.date.today() #giorno attuale
        driver = preparaDriver()
        dataOdierna=date.strftime("%d/%m/%Y")
        for city in cities:
            parametri = { # parametri vuoti per prendere le strutture totali disponibili indipendentemente dal giorno
                'ss': '',
                'dest_type': 'city'
            }
            parametri['ss'] = city
            queryString = urlencode(parametri) #libreria urlLib per formattare la url
            url="https://www.booking.com/searchresults.html?%s" % queryString
            driver.get(url)
            #driver = preparaDriver(url)
            stringaCitta=city #variabile da assegnare alla lista dei campi_alloggi
            xpathStruttura = driver.find_element_by_xpath('//*[@id="right"]/div[5]/div/div[1]/div/h1').text  #percorso del titolo che varia a seconda delle citta
            numstruttureTrovate=(re.findall('[\d]*[\.]?[\d]+',xpathStruttura)[0]) #prende il primo numero in lista relativo alle strutture
            stringaCitta=stringaCitta.split(",")[0]
            wait = WebDriverWait(driver, attesaCorta).until( #attesa esplicita
                EC.presence_of_all_elements_located(
                    (By.CLASS_NAME, 'sr-hotel__title')))
            alloggiCitta.append(scrapeNumeroStruttureDisponibili(driver, dataOdierna, numstruttureTrovate))
            
        keys = alloggiCitta[0].keys()
        with open(dataOdierna.replace('/','-')+'_disponibilitaOdiernaBooking.csv', 'a', newline='') as csvFile: #appende ai dati precedenti 
            dict_writer = csv.DictWriter(csvFile, keys) # imposto i nomi dei campi
            dict_writer.writeheader()
            dict_writer.writerows(alloggiCitta)
    
    finally:
        driver.close()
        driver.quit()

        

L'Aquila
83
Teramo
20
Pescara
239
Chieti
60
Potenza
65
Matera
717
Cosenza
71
Catanzaro
57
Crotone
124
Vibo Valentia
35
Reggio Calabria
223
Caserta
163
Benevento
64
Avellino
23
Salerno
682
Napoli
4.013
Piacenza
54
Parma
252
Reggio Emilia
71
Modena
126
Ferrara
221
Ravenna
158
Bologna
1.431
Rimini
1.025
Forlì
45
Udine
87
Gorizia
28
Trieste
524
Pordenone
29
Viterbo
207
Rieti
40
Latina
35
Frosinone
25
Roma
10.725
Imperia
300
Savona
112
La Spezia
819
Genova
876
Bergamo
437
Brescia
123
Como
538
Cremona
58
Lecco
105
Lodi
24
Mantova
127
Milano
3.653
Varese
78
Monza
60
Sondrio
25
Pavia
90
Ancona
140
Macerata
33
Ascoli Piceno
106
Fermo
54
Pesaro
141
Campobasso
77
Isernia
16
Asti
111
Alessandria
55
Biella
42
Vercelli
24
Novara
43
Cuneo
75
Torino
1.254
Verbania
205
Foggia
99
Taranto
261
Brindisi
122
Lecce
783
Bari
845
Andria
41
Sassari
151
Nuoro
40
Oristano
73
Cagliari
827
Carbonia
31
Trapani
521
Agrigento
366
Caltanissetta
65
Enna
59
Ragusa
382
Siracusa
1.486
Palermo
2.168
Messina
207
Catania
1.65